In [1]:
path_file = "../data/COLIEE2024statute_data-English/fewshot/"

In [2]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
        tree = Et.parse(filexml)
        root = tree.getroot()
        samples = []
        for i in range(0, len(root)):
            sample = {'result': []}
            for j, e in enumerate(root[i]):
                if e.tag == "t1":
                    sample['result'] = format_first_line(e.text.strip())
                elif e.tag == "t2":
                    question = e.text.strip()
                    sample['content'] = question if len(question) > 0 else None
            sample.update(
                {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
            # filter the noise samples
            if sample['content'] is not None:
                samples.append(sample)
            else:
                print("[Important warning] samples {} is ignored".format(sample))
        return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

In [4]:
from transformers import AutoTokenizer, BloomForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xxl"
cache_dir = ".cache"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
		model_name, device_map="auto", cache_dir=cache_dir, torch_dtype=torch.float16, load_in_8bit=True
	)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:09<00:00, 13.92s/it]


In [25]:
from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip()#.lower()

def prompting(premise, hypothesis, label, template=None):
    if "true" in template.lower():
        answer = "True"
        if "N" in label:
            answer = "False"
    else:
        answer = "Yes"
        if "N" in label:
            answer = "No"
    text = template.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis)
    return text+"\nLet't think step by step "

def write_cot(result, filename):
    data = [ujson.dumps(line, escape_forward_slashes=False) for line in result]
    with open(filename, "w", encoding="utf-8") as f:
        for line in data:
            f.write(line+'\n')

In [26]:
list_prompt = readfile("../data/prompt4.json")
list_prompt

[{'id': 19,
  'label': ['true', 'false'],
  'prompt': 'According to the given legal reasoning approach.\nDocument: {{premise}}\nQuestion: {{hypothesis}}? '}]

In [27]:
import copy

def predict(model, tokenizer, files=["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], output="../output/cot/newpromt_"):
    for file in files:
        test_file = path_file+file+".xml"
        data = load_samples(test_file)
        
        acc = {}
        for template_prompt in list_prompt:
            template_prompt = template_prompt["prompt"]
            # print(template_prompt)
            result = []
            count = 0
            for item in tqdm(data):
                label = item["label"]
                hypothesis = item["content"]
                premise = item["result"]
                #Important: You must use dot-product, not cosine_similarity
                text = prompting(premise, hypothesis, label, template_prompt)
                inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
                outputs = model.generate(inputs, max_new_tokens=256)
                output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
                item.update({"prompt": text})
                item.update({"cot": output_text})
                result.append(item)
                if count < 5:
                    print(text)
                    print(output_text)
                    count += 1
        #     acc.update({template_prompt: count/len(data)})
        # write_cot(result, output+file+"_cot.jsonl")

In [28]:
predict(model, tokenizer, ["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], "../output/generated_cot/")

  1%|█▍                                                                                                                                                         | 1/111 [00:05<09:28,  5.16s/it]

According to the given legal reasoning approach.
Document: Article 5
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
(3) Notwithstanding the provisions of paragraph (1), a minor may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose. The same applies if the minor disposes of property that the legal representative has permitted the minor to dispose of without specifying a purpose.
Question: A contract of sales concluded by a minor may not be rescinded if it relates to daily life, even in cases the consent of the parental authority is not obtained.? 
Let't think step by step 
The contract of s

  2%|██▊                                                                                                                                                        | 2/111 [00:10<09:41,  5.33s/it]

According to the given legal reasoning approach.
Document: Article 9
A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Question: A contract of sales concluded by an adult ward may be rescinded except relating to daily life, even in cases the consent of the adult guardian has been obtained, the contract? 
Let't think step by step 
The contract of sales concluded by an adult ward may be rescinded except relating to daily life, even in cases the consent of the adult guardian has been obtained, the contract? The answer: yes.


  3%|████▏                                                                                                                                                      | 3/111 [00:38<28:28, 15.82s/it]

...

  4%|██████▏                                                                                                                                                    | 4/101 [00:26<11:01,  6.82s/it]

According to the given legal reasoning approach.
Document: Article 9 A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Question: If an adult ward acquires land by gift, the guardian of the adult ward may not rescind the gift.? 
Let't think step by step 
The act of an adult ward is voidable. The act of an adult ward is voidable if it is a juridical act. The act of an adult ward is voidable if it is a juridical act. The answer: no.


  5%|███████▋                                                                                                                                                   | 5/101 [00:31<09:56,  6.22s/it]

According to the given legal reasoning approach.
Document: Article 9 A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Article 102 An act that a person with qualified legal capacity performs as an agent of another person may not be rescinded on the grounds of qualified legal capacity;provided, however, that this does not apply to an act performed by a person with qualified legal capacity as a legal representative of another person with qualified legal capacity.
Question: An act that A, an adult ward, performs as legal representative of B, a minor, may not be rescinded on the grounds of qualified legal capacity of A.? 
Let't think step by step 
A, an adult ward, performs as legal representative of B, a minor, may not be rescinded on the grounds of qualified legal capacity of A. Therefore, the answer is no.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [12:57<00:00,  7.70s/it]


In [32]:
from os import listdir
from os.path import isfile, join

myfiles = [f.replace(".xml", "") for f in listdir(path_file) if isfile(join(path_file, f))]
# myfiles

['riteval_H18_en',
 'riteval_H30_en',
 'riteval_H29_en',
 'riteval_H25_en',
 'riteval_H22_en',
 'riteval_H21_en',
 'riteval_R04_en',
 'riteval_H24_en',
 'riteval_H28_en',
 'riteval_H20_en',
 'riteval_R01_en',
 'riteval_R02_en',
 'riteval_H26_en',
 'riteval_R03_en',
 'riteval_H23_en',
 'riteval_H27_en',
 'riteval_H19_en']